<a href="https://colab.research.google.com/github/lijishan/chinese_clip/blob/master/chinese_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/OFA-Sys/Chinese-CLIP.git

Cloning into 'Chinese-CLIP'...
remote: Enumerating objects: 1459, done.
remote: Counting objects: 100% (659/659), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 1459 (delta 563), reused 484 (delta 453), pack-reused 800
Receiving objects: 100% (1459/1459), 2.59 MiB | 11.40 MiB/s, done.
Resolving deltas: 100% (962/962), done.


# Install requirements

In [2]:
!pip install cn_clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.5/306.5 kB 16.1 MB/s eta 0:00:00


In [3]:
!pip install numpy
!pip install numpy
!pip install tqdm
!pip install six
!pip install timm
!pip install lmdb==1.3.0
!pip install torch>=1.7.1
!pip install torchvision

测试

In [4]:
import torch
from PIL import Image

import cn_clip.clip as clip
from cn_clip.clip import load_from_name, available_models
print("Available models:", available_models())
# Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = load_from_name("ViT-B-16", device=device, download_root='./')
model.eval()
image = preprocess(Image.open("examples/pokemon.jpeg")).unsqueeze(0).to(device)
text = clip.tokenize(["杰尼龟", "妙蛙种子", "小火龙", "皮卡丘"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    # 对特征进行归一化，请使用归一化后的图文特征用于下游任务
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    logits_per_image, logits_per_text = model.get_similarity(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # [[1.268734e-03 5.436878e-02 6.795761e-04 9.436829e-01]]

Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']


100%|███████████████████████████████████████| 718M/718M [01:24<00:00, 8.95MiB/s]


Loading vision model config from /usr/local/lib/python3.10/dist-packages/cn_clip/clip/model_configs/ViT-B-16.json
Loading text model config from /usr/local/lib/python3.10/dist-packages/cn_clip/clip/model_configs/RoBERTa-wwm-ext-base-chinese.json
Model info {'embed_dim': 512, 'image_resolution': 224, 'vision_layers': 12, 'vision_width': 768, 'vision_patch_size': 16, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 12, 'text_type_vocab_size': 2}
Label probs: [[1.291e-03 5.490e-02 6.909e-04 9.434e-01]]
